In [2]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import caffe
from nbfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())
from generate_prototxt_files import *
from os.path import join, basename

In [3]:
def get_cl_args(cl_args):



    if any(["jupyter" in arg for arg in sys.argv]):
        sys.argv=sys.argv[:1]

    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    for k,v in cl_args.iteritems():
        parser.add_argument('--' + k, type=type(v), default=v, help=k)

    args = parser.parse_args()
    cl_args.update(args.__dict__)
    return cl_args

In [ ]:
def save_plot(cl_args):
    suffix = "learning_curve_lr_%8.6f_%s.jpg"%(cl_args["lr"],basename(cl_args["solver_path"]))
    plt.figure(1)
    plt.plot([losses[str(epoch) + "_mean"]["final_loss"] for epoch in range(ep + 1)])
    #plt.show()
    plt.savefig(join(cl_args["save_dir"],"tr_" + suffix))
    

In [ ]:
def write_epoch_losses(losses, loss_keys,ep):
    losses[str(ep) + "_mean"] = {k:np.mean(losses[ep][k]) for k in loss_keys}
    for k in loss_keys:
        loss = losses[str(ep) + "_mean"][k]
        sys.stderr.write("\n Epoch %i: Final %s Loss = %6.3f\n" % (ep, k, loss))
        sys.stderr.flush()
    return losses

In [ ]:
def write_iteration_losses(losses, solver,loss_keys,it):
    for k in loss_keys:
        loss = np.float32(solver.net.blobs[k].data)
        losses[ep][k].append(loss)
        sys.stderr.write("\n Loss at iteration %i for %s is %8.4f " % (it,k,loss))
        sys.stderr.flush()
    return losses

In [ ]:
def setup_losses

In [ ]:
def do_train(solver, cl_args):
    num_tr = 32 #get_num_examples(join(cl_args["proto_basepath"], cl_args["train_proto_name"]))
    tr_its_per_epoch = int(float(num_tr) / cl_args["tr_batch_size"])
    #val_its_per_epoch = int(float(num_val) / val_batch_size)
    num_epochs = cl_args["num_epochs"]

    loss_keys = ["L_cls","L_obj","L_xy","L_wh","L_rec","final_loss"]
    losses = {}
    
    for ep in range(num_epochs):
        losses[ep] = {k:[] for k in loss_keys}
        losses[str(ep) + "_mean"] = {k:0 for k in loss_keys}
        for it in range(tr_its_per_epoch):
            solver.step(1)
            
            losses = write_iteration_losses(losses, solver,loss_keys,it)
            
        losses = write_epoch_losses(losses, loss_keys,ep)

        save_plot(cl_args)
        


In [6]:
if __name__ == "__main__":

    cl_args = {"solver_path": "/project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup/really_small_test/node_49_solver_vanilla.prototxt" }
    cl_args = get_cl_args(cl_args)
    solver = caffe.SGDSolver(cl_args["solver_path"])
    solver.step(1)
    #do_train(solver, cl_args)
    
    

In [3]:
solver.step(1)